In [ ]:
# выбор видеокарты
# оставляет видимой только одну GPU 
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import torch
# проверка на количество видеокарт (если 1 то все ок, если 2 то надо смотреть)
print('CUDA_VISIBLE_DEVICES', torch.cuda.device_count())
print('CURRENT DEVICE', torch.cuda.current_device())
print('____________________________')

In [ ]:
print(torch.cuda.memory_summary())

In [ ]:
import torch

from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

import pandas as pd

In [ ]:
# CPU or GPU
usegpu = True

In [ ]:
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device='cuda')
# model = SentenceTransformer('all-MiniLM-L6-v2', device='cuda')
model = SentenceTransformer('paraphrase-MiniLM-L3-v2', device='cuda')


In [ ]:
# открыть файл
# id - id файла
# folder - путь к папке ('text')
# на ВЫХОД:
# text_in_string текст в формате строки
def open_file(d_id, folder):
    path = folder + '/' + str(d_id) + '.txt'
    text_file = open(path,'r')
    text_in_string = text_file.read()
    text_file.close()
    return text_in_string

In [ ]:
# amount_label - сколько экземпляров 0 и сколько экземпляров 1 надо добавить в тренировочную базу
def create_data_for_training_sbert(path_dev_file, texts_folder, amount_label):
    df = pd.read_csv(path_dev_file)
    df0 = df[df['label'] == 0]
    df1 = df[df['label'] == 1]
    train_examples = []
    print("Text preprocessing")
    counter = 0
    for i in range(0, len(df0), len(df0)//amount_label):
        if len(train_examples) >= amount_label:
            break
        doc_id_1 = df0.take([i]).values[0][0] #doc_id_1
        doc_id_2 = df0.take([i]).values[0][2] #doc_id_2
        label1 = df0.take([i]).values[0][4] #label
        
        text1 = open_file(doc_id_1, texts_folder)

        text2 = open_file(doc_id_2, texts_folder)
        
        elem = InputExample(texts=[text1, text2], label=float(label1))

        train_examples.append(elem)
        
        counter += 1
        print('\r', 'iter... {} / {}'.format(str(counter), amount_label*2), end='          ')
        
    
    for i in range(0, len(df1), len(df1)//amount_label):
        if len(train_examples) >= amount_label*2:
            break
        doc_id_1 = df1.take([i]).values[0][0] #doc_id_1
        doc_id_2 = df1.take([i]).values[0][2] #doc_id_2
        label1 = df1.take([i]).values[0][4] #label
        
        text1 = open_file(doc_id_1, texts_folder)

        text2 = open_file(doc_id_2, texts_folder)
        
        elem = InputExample(texts=[text1, text2], label=float(label1))

        train_examples.append(elem)

        counter += 1
        print('\r', 'iter... {} / {}'.format(str(counter), amount_label*2), end='          ')
    
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
    
    return train_dataloader

### Предобработка

In [ ]:
# dataset_v1
# path_dev_file = '../data/dataset_v1/dev.csv'
# path_train_file = '../data/dataset_v1/train.csv'
# texts_folder = '../data/dataset_v1/text'

# SimRuWiki
# path_dev_file = '../data/SimRuWiki/dev.csv'
# path_train_file = '../data/SimRuWiki/train.csv'
# texts_folder = '../data/SimRuWiki/texts'

# SimEnWiki
path_dev_file = '../data/SimEnWiki/dev.csv'
path_train_file = '../data/SimEnWiki/train.csv'
texts_folder = '../data/SimEnWiki/texts'

# dev_df = pd.read_csv(path_dev_file)
# train_dl = create_data_for_training(path_train_file, texts_folder)
# train_dl = create_data_for_training(path_dev_file, texts_folder)

amount_label = 25000
train_dl = create_data_for_training_sbert(path_train_file, texts_folder, amount_label)
# train_dl = create_data_for_training1(path_train_file, texts_folder, amount_label)

In [ ]:
len(train_dl)
# количество экземпляров / количество батчей

In [ ]:
type(train_dl)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
num_epoch = 3
warmup_steps = 100
model_save_path = '../models/sbert-50000-epoch3-batch16-warmupcosine-model3'

In [ ]:
model.fit(train_objectives=[(train_dl, train_loss)],
          epochs=num_epoch, 
          scheduler='warmupcosine',
          warmup_steps=warmup_steps,
          output_path=model_save_path)

In [ ]:
model = SentenceTransformer('/workspace/src/models/sbert-50000-epoch3-batch16-warmupcosine-model3', device='cuda')

In [ ]:
# SimRuWiki
path_dev_file = '../data/SimRuWiki/dev.csv'
path_train_file = '../data/SimRuWiki/train.csv'
texts_folder = '../data/SimRuWiki/texts'

In [ ]:
amount_label = 25000
train_dl = create_data_for_training_sbert(path_train_file, texts_folder, amount_label)

In [ ]:
len(train_dl)

In [ ]:
train_loss = losses.CosineSimilarityLoss(model)

In [ ]:
num_epoch = 3
warmup_steps = 100
model_save_path = '../models/sbert-50000en-50000ru-epoch3-batch16-warmupcosine-model3'

In [ ]:
model.fit(train_objectives=[(train_dl, train_loss)],
          epochs=num_epoch, 
          scheduler='warmupcosine',
          warmup_steps=warmup_steps,
          output_path=model_save_path)